In [66]:
from Bio.Alphabet import generic_protein, generic_dna
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.SeqIO import to_dict, parse
from os import listdir, mkdir
from os.path import isdir

def _sequence_back_translate(aligned_protein_record, unaligned_nucleotide_record, gap=None):
    #TODO - Separate arguments for protein gap and nucleotide gap?
    if hasattr(aligned_protein_record.seq.alphabet, "gap_char"):
        if not gap:
            gap = aligned_protein_record.seq.alphabet.gap_char
        elif gap != aligned_protein_record.seq.alphabet.gap_char:
            raise ValueError("Gap %r does not match %r from %r aligned protein alphabet" \
                             % (gap, aligned_protein_record.seq.alphabet.gap_char,
                                aligned_protein_record.id))
    if not gap:
        raise ValueError("Please supply the protein alignment gap character")

    alpha = unaligned_nucleotide_record.seq.alphabet
    if hasattr(alpha, "gap_char"):
        gap_codon = alpha.gap_char * 3
        assert len(gap_codon) == 3
    else:
        from Bio.Alphabet import Gapped
        alpha = Gapped(alpha, gap)
        gap_codon = gap*3

    seq = []
    nuc = str(unaligned_nucleotide_record.seq)
    for amino_acid in aligned_protein_record.seq:
        if amino_acid == gap:
            seq.append(gap_codon)
        else:
            seq.append(nuc[:3])
            nuc = nuc[3:]
    assert not nuc, "Nucleotide sequence for %r longer than protein %s" \
        % (unaligned_nucleotide_record.id, aligned_protein_record.id)

    aligned_nuc = unaligned_nucleotide_record[:] #copy for most annotation
    aligned_nuc.letter_annotation = {} #clear this
    aligned_nuc.seq = Seq("".join(seq), alpha) #replace this
    if len(aligned_protein_record.seq) * 3 == len(aligned_nuc):
        return aligned_nuc
    else:
        print('--->',len(aligned_protein_record.seq) * 3, len(aligned_nuc))
        for i in range(len(aligned_protein_record.seq) * 3-len(aligned_nuc)):
            aligned_nuc += '-'
    return aligned_nuc

def alignment_back_translate(protein_alignment, nucleotide_records, key_function=None, gap=None):
    #TODO - Separate arguments for protein and nucleotide gap characters?
    if key_function is None:
        key_function = lambda x: x

    if hasattr(protein_alignment._alphabet, "gap_char"):
        if not gap:
            gap = protein_alignment._alphabet.gap_char
        elif gap != protein_alignment._alphabet.gap_char:
            raise ValueError("Gap %r does not match %r from protein alignment alphabet" \
                             % (gap, protein_alignment._alphabet.gap_char))

    aligned = []
    for protein in protein_alignment:
        try:
            nucleotide = nucleotide_records[key_function(protein.id)]
        except KeyError:
            print(str(protein.id))
            raise ValueError("Could not find nucleotide sequence for protein %r" % (protein.id))
        temp = _sequence_back_translate(protein, nucleotide, gap)
        aligned.append(temp)
    A = set([])
    for a in aligned:
        A.add(len(a.seq))
    print('Done')
    print(A)
    return MultipleSeqAlignment(aligned)

def Parse_Nuc_Sequences(seq_path):
    fasta_sequenes = parse(open(seq_path),'fasta')
    sequences = []
    try:
        for f in fasta_sequenes:
            temp = SeqRecord(Seq(str(f.seq)[:-2], generic_dna), id=str(f.name))
            sequences.append(temp)
    except UnicodeDecodeError:
        pass
    return to_dict(sequences)

def Parse_MSA(seq_path):
    fasta_sequenes = parse(open(seq_path),'fasta')
    sequences = []
    for f in fasta_sequenes:
        temp = SeqRecord(Seq(str(f.seq), generic_protein), id=str(f.name))
        sequences.append(temp)
    return MultipleSeqAlignment(sequences)

def Write_Nuc_MSA(nuc_alignments, op_filepath):
    fbuf = open(op_filepath,'w')
    for n in nuc_alignments:
        fbuf.write('>'+str(n.id)+'\n')
        fbuf.write(str(n.seq)+'\n')
    fbuf.close()
    

In [67]:
dna_seq_path = '/Users/harihara/Mount/CMSC829A/Data/Accessory_Genes_Nucleotides/'
dna_seq_files = listdir(dna_seq_path)
prot_seq_path = '/Users/harihara/Mount/CMSC829A/Data/Accessory_Genes_Proteins_MSA/'
out_path = '/Users/harihara/Mount/CMSC829A/Data/Accessory_Genes_Nucleotides_MSA/'
if not isdir(out_path):
    mkdir(out_path)

In [68]:
ctr = 0
dna_seq_files = ['copA.fna']
for seq_path in dna_seq_files:
    if(seq_path.startswith(".")):
        continue
    
    dna_seqs = Parse_Nuc_Sequences(dna_seq_path+seq_path)
    prot_msa = Parse_MSA(prot_seq_path+seq_path.replace(".fna",".faa"))
    nuc_msa = alignment_back_translate(prot_msa, dna_seqs,gap = "-")
    Write_Nuc_MSA(nuc_msa, out_path+seq_path)
    print(ctr,seq_path)
    #except AssertionError:
    #    print('--->',ctr,seq_path)
    #except ValueError:
    #    print('+++>',ctr,seq_path)
    ctr += 1

AssertionError: Nucleotide sequence for 'GCF_000005845.2_ASM584v2.0' longer than protein GCF_000005845.2_ASM584v2.0